In [437]:
import pandas as pd
from pivottablejs import pivot_ui
# Pandas options
pd.options.display.max_rows = 999
pd.set_option('expand_frame_repr', False)
# jupyter options
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline


In [438]:
def _read_and_stack(file):
    """ Reads the data and format it in long format with .stack() with countries as index 

    Parameters
    ----------
    file: String
        the file to import, from the COVID-19 repository on github:
        https://github.com/CSSEGISandData/COVID-19.git
        
    Returns
    -------
    - df:
        pandas dataframe, stacked, with coutries as index
    
    See Also
    --------

    Examples
    --------
    confirmed = _read_and_stack("Confirmed")

    """
    
    df = (pd
          .read_csv(rf"csse_covid_19_data\csse_covid_19_time_series\time_series_19-covid-{file}.csv")
          .reset_index()
          .drop(["index", "Lat", "Long"], axis=1)
          .set_index(['Province/State', 'Country/Region'])
          .stack()
         )
    df.index.set_names(['Province/State', 'Country/Region', "Date"], inplace=True)
    
    return df

In [439]:
# Datframe with all the data
files=["Confirmed", "Deaths", "Recovered"]
df = pd.DataFrame()
for file in files:
    df[file] = _read_and_stack(file)

# cleaning up the data
df = df.reset_index()
# df.loc[df['Province/State'].isnull(),'Province/State'] = df['Country/Region']
df['Province/State'] = df['Province/State'].fillna(df['Country/Region'])

# Add the mortality rate, number of new confirmed and number of new deaths
df["rate"] = (df.Deaths / df.Confirmed).fillna(0)
df["New Confirmed"] = df.groupby(['Country/Region', "Province/State"]).Confirmed.diff().fillna(0)
df["New Deaths"] = df.groupby(['Country/Region', "Province/State"]).Deaths.diff().fillna(0)
# df.loc[df["Country/Region"] == "China"]

In [440]:
# Export the interactive table / chart
pivot_ui(df,outfile_path="time_series_pivotchart.html")

In [323]:
df.groupby(df.index.names)["Confirmed"]